In [239]:
import pandas as pd 
import os 

from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

import seaborn as sns

import datasets
from datasets import load_dataset, load_from_disk
import numpy as np
import json

In [200]:
pwd

'/mnt/home/al2644/research/projects/perturb-r/notebook/deepmath'

In [203]:
cd /mnt/home/al2644/research/projects/

/mnt/home/al2644/research/projects


# Check generated sequences

In [246]:
train_df = pd.read_parquet("rlvr/data/train/deepmath_level3-4-stage2-post-step285-reward-lt-1.0/train.parquet")

In [236]:
ez_train_df = train_df.apply(lambda x: x['reward_model']['ground_truth'].lower() in ['yes', 'no', 'true', 'false', 'a', 'b', 'c', 'd', 'e'], axis = 1)
ez_train_df = train_df[ez_train_df]
               

In [238]:
ez_train_df[~ez_train_df['zero_success']]

,data_source,prompt,ability,reward_model,extra_info,zero_success
14,deepmath,"[{'content': 'In Category Theory, can an objec...",math,"{'ground_truth': 'Yes', 'style': 'rule'}","{'difficulty': 3.0, 'index': 14, 'split': 'tra...",False
33,deepmath,"[{'content': 'Let $(u_{n})_{n},(v_{n})_{n}$ be...",math,"{'ground_truth': 'True', 'style': 'rule'}","{'difficulty': 3.0, 'index': 33, 'split': 'tra...",False
181,deepmath,[{'content': 'If we write the 10 numerals in a...,math,"{'ground_truth': 'Yes', 'style': 'rule'}","{'difficulty': 3.0, 'index': 181, 'split': 'tr...",False
217,deepmath,[{'content': 'Determine if the even prime numb...,math,"{'ground_truth': 'No', 'style': 'rule'}","{'difficulty': 3.0, 'index': 217, 'split': 'tr...",False
253,deepmath,"[{'content': 'In neutral geometry, if opposite...",math,"{'ground_truth': 'B', 'style': 'rule'}","{'difficulty': 3.0, 'index': 253, 'split': 'tr...",False
...,...,...,...,...,...,...
12176,deepmath,[{'content': 'In a non-cyclic group of 9 eleme...,math,"{'ground_truth': 'Yes', 'style': 'rule'}","{'difficulty': 4.5, 'index': 12176, 'split': '...",False
12179,deepmath,[{'content': 'Determine if the following manip...,math,"{'ground_truth': 'Yes', 'style': 'rule'}","{'difficulty': 4.5, 'index': 12179, 'split': '...",False
12182,deepmath,[{'content': 'Given that $E[X]=\theta(n^{1.5})...,math,"{'ground_truth': 'D', 'style': 'rule'}","{'difficulty': 4.5, 'index': 12182, 'split': '...",False
12185,deepmath,"[{'content': 'In triangle \(ABC\), \(BR = RC\)...",math,"{'ground_truth': 'E', 'style': 'rule'}","{'difficulty': 4.5, 'index': 12185, 'split': '...",False


# Generate & Inject random thoughts

In [21]:
tokenzier = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

In [55]:
prompt_str = tokenzier.apply_chat_template(
    [
        {'role': 'system', 'content': 'You are taking the exam at International Mathematical Olympiad.'},
        {'role': 'user', 'content': 'You need to solve this very difficult algebra problem. Think very carefully and exhaustively about this problem'}
    ],
    tokenize=False,
    add_generation_prompt=True
)

In [212]:
random_thoughts_df[random_thoughts_df['think_tokens'] > 2048]

,random_thought,prompt,think_tokens
1,"Okay, so I have this calculus problem to solve...",<｜begin▁of▁sentence｜>You are the smartest math...,2104
5,"Okay, so I've got this Discrete Mathematics pr...",<｜begin▁of▁sentence｜>You are the smartest math...,5991
13,"Alright, I've got this differential equation t...",<｜begin▁of▁sentence｜>You are the smartest math...,4300
15,"Okay, so I've been given this problem that req...",<｜begin▁of▁sentence｜>You are the smartest math...,3560
16,"Okay, so I have this problem in front of me ab...",<｜begin▁of▁sentence｜>You are the smartest math...,2122
...,...,...,...
9974,"Okay, so I need to solve a differential equati...",<｜begin▁of▁sentence｜>You are the smartest math...,2620
9975,"Okay, so I've got this Algebra problem here th...",<｜begin▁of▁sentence｜>You are the smartest math...,3781
9980,"Okay, so I've just realized I need to solve th...",<｜begin▁of▁sentence｜>You are the smartest math...,2288
9981,"Okay, so I came across this differential equat...",<｜begin▁of▁sentence｜>You are the smartest math...,3160


## Inject Thoughts

In [262]:
path="/mnt/home/al2644/research/projects/perturb-r/results/deepmath_generate_data/inject_thoughts/Qwen2.5-1.5B-DeepMath-stage1-grpo-level3-4-5epochs-4rollouts-8192max-length-5epochs.pickle"
df = pd.read_pickle(path)

In [279]:
example = df.iloc[0]
conversations = [{'role': 'user', 'content': example['problem']}]

In [267]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

In [284]:
filepath = "/mnt/home/al2644/research/projects/rlvr/data/train/deepmath_level3-4-stage2-post-step285-reward-lt-1.0-randomthought/train.parquet"
random_thought_train_df = pd.read_parquet(filepath)

In [294]:
random_thought_train_df['prompt'].iloc[1]

array([{'content': 'Determine the number of points in the interval \\([0, 2\\pi]\\) where the function \\( g(x) = \\left[\\cos x\\right] \\) is discontinuous. Here, \\([\\cdot]\\) denotes the greatest integer function.', 'role': 'user'}],
      dtype=object)

In [295]:
chat = random_thought_train_df['prompt'].iloc[1]
has_assistant_message = any(message.get("role") == "assistant" for message in chat)

In [296]:
prompt_w_template = tokenizer.apply_chat_template(chat, add_generation_prompt=not has_assistant_message, tokenize=False).removesuffix(tokenizer.eos_token)




In [297]:
print(prompt_w_template)

<｜begin▁of▁sentence｜><｜User｜>Determine the number of points in the interval \([0, 2\pi]\) where the function \( g(x) = \left[\cos x\right] \) is discontinuous. Here, \([\cdot]\) denotes the greatest integer function.<｜Assistant｜><think>



# Rollouts

In [119]:
rollout_dataset = load_dataset('aochongoliverli/R1-Distill-Qwen-1.5B-DeepMath-stage1-grpo-level3-4-5epochs-4rollouts-8192max-length-rollouts')['train']

In [156]:
import numpy as np
from datasets import Dataset   # only needed for context

def post_cutoff_reward(example, *, cutoff_step: int = 300):
    """
    Mean reward for timesteps > cutoff_step.
    Returns a dict so 🤗 Datasets adds it as a new column.
    """
    rewards = np.asarray(example["reward"])
    steps   = np.asarray(example["global_step"])
    mask    = steps > cutoff_step
    return {
        "epoch4_reward": rewards[mask].mean() if mask.any() else np.nan
    }

# add the column in one pass (rollout_dataset is mutated in-place unless you copy)
rollout_dataset = rollout_dataset.map(
    post_cutoff_reward,
    fn_kwargs={"cutoff_step": 285},
    num_proc=4,
    desc="computing mean reward after step 285"
)


computing mean reward after step 285 (num_proc=4):   0%|          | 0/12197 [00:00<?, ? examples/s]

In [160]:
sample_dataset = rollout_dataset.filter(lambda x : x['epoch4_reward'] <1.)

Filter:   0%|          | 0/12197 [00:00<?, ? examples/s]

In [181]:
sample_dataset

Dataset({
    features: ['index', 'question', 'response', 'reward', 'global_step', 'post300_reward', 'epoch4_reward'],
    num_rows: 6578
})

In [187]:
len(tokenzier.encode(noise))

2586

In [188]:
print('acc: ', example['epoch4_reward'])
print(prompt + noise)

acc:  0.875
<｜begin▁of▁sentence｜><｜User｜>Evaluate the integral: \[ \int_{0}^{\sqrt[6]{\pi}} x^5 \cos(x^6) \, dx \]<｜Assistant｜><think>
Okay, so I have this problem in front of me about an object moving on a parabolic surface. Let me see if I can make sense of it. The problem says that an object is moving along a smooth parabolic path, and after moving a small distance ds along the surface, its weight increases by a small amount dw, and the increase dw is proportional to the distance moved ds. The goal is to find the ratio k = dw/ds.

Alright, first, I need to visualize the parabolic path. It's a standard parabola, probably of the form y = ax², right? Or maybe it's better to express it in terms of y as a function of x or vice versa. The slope of the parabola will be important because weight is related to the component of gravitational acceleration along the surface.

So, when an object is on a path, the total gravitational force, which is mg, gets split into two components: one perpendi

In [185]:
print(example['response'][4])

Okay, so I have this integral to solve: the integral from 0 to the sixth root of pi of x^5 times cosine of x^6, all with respect to x. Hmm, that's \[ \int_{0}^{\sqrt[6]{\pi}} x^5 \cos(x^6) dx \]. Alright, let me see how to approach this.

I remember that when I see something like x^n times cosine of x^m dx, sometimes a substitution might help. Because in this case, the exponent on x in cosine is x^6, and the other part is x^5. Maybe if I let u be x^6, that could simplify things. Let me try that.

So, if I set u = x^6, then du/dx would be 6x^5, right? That means du = 6x^5 dx. Wait, so x^5 dx is du/6. That seems useful because I have an x^5 dx term in the integral. So, if I do that substitution, I can rewrite the integral in terms of u.

Let me write that down more clearly. Substitute:

When x = 0, u = 0^6 = 0.

When x = \sqrt[6]{\pi}, u = (\sqrt[6]{\pi})^6 = pi.

So, the limits of integration become from 0 to pi. That seems simpler.

So, substituting into the integral, we have:

Integra